# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [2]:
# Your solution goes here

import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,'
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
response = requests.get(url, headers=headers)

In [3]:
soup = BeautifulSoup(response.content, "html.parser")

In [4]:
#ipc-title__text
movie_title = soup.find_all(class_ = "ipc-title__text")
movie_title

[<h1 class="ipc-title__text">Advanced search</h1>,
 <h3 class="ipc-title__text">1. El silencio de los corderos</h3>,
 <h3 class="ipc-title__text">2. Uno de los nuestros</h3>,
 <h3 class="ipc-title__text">3. Terminator 2: El juicio final</h3>,
 <h3 class="ipc-title__text">4. Mi primo Vinny</h3>,
 <h3 class="ipc-title__text">5. Esencia de mujer</h3>,
 <h3 class="ipc-title__text">6. Reservoir Dogs</h3>,
 <h3 class="ipc-title__text">7. Tomates verdes fritos</h3>,
 <h3 class="ipc-title__text">8. El último mohicano</h3>,
 <h3 class="ipc-title__text">9. Bailando con lobos</h3>,
 <h3 class="ipc-title__text">10. Desafío total</h3>,
 <h3 class="ipc-title__text">11. Algunos hombres buenos</h3>,
 <h3 class="ipc-title__text">12. Solo en casa</h3>,
 <h3 class="ipc-title__text">13. Sin perdón</h3>,
 <h3 class="ipc-title__text">14. Thelma &amp; Louise</h3>,
 <h3 class="ipc-title__text">15. Eduardo Manostijeras</h3>,
 <h3 class="ipc-title__text">16. La bella y la bestia</h3>,
 <h3 class="ipc-title__tex

In [5]:
len(movie_title)

27

In [6]:
movie_title[1]

<h3 class="ipc-title__text">1. El silencio de los corderos</h3>

In [7]:
print(movie_title[1].text)

1. El silencio de los corderos


In [8]:
movie_title[1].getText().strip()

'1. El silencio de los corderos'

In [9]:
movies_names = [movie.text.strip() for movie in movie_title]
movies_names

['Advanced search',
 '1. El silencio de los corderos',
 '2. Uno de los nuestros',
 '3. Terminator 2: El juicio final',
 '4. Mi primo Vinny',
 '5. Esencia de mujer',
 '6. Reservoir Dogs',
 '7. Tomates verdes fritos',
 '8. El último mohicano',
 '9. Bailando con lobos',
 '10. Desafío total',
 '11. Algunos hombres buenos',
 '12. Solo en casa',
 '13. Sin perdón',
 '14. Thelma & Louise',
 '15. Eduardo Manostijeras',
 '16. La bella y la bestia',
 '17. Misery',
 '18. Los chicos del barrio',
 '19. El padrino parte III',
 '20. Aladdín',
 '21. La caza del Octubre Rojo',
 '22. JFK: Caso abierto',
 '23. Glengarry Glen Ross: Éxito a cualquier precio',
 '24. Braindead: tu madre se ha comido a mi perro',
 '25. Despertares',
 'Recently viewed']

In [10]:
movies_names_filtered = movies_names[1:-1]
movies_names_filtered

['1. El silencio de los corderos',
 '2. Uno de los nuestros',
 '3. Terminator 2: El juicio final',
 '4. Mi primo Vinny',
 '5. Esencia de mujer',
 '6. Reservoir Dogs',
 '7. Tomates verdes fritos',
 '8. El último mohicano',
 '9. Bailando con lobos',
 '10. Desafío total',
 '11. Algunos hombres buenos',
 '12. Solo en casa',
 '13. Sin perdón',
 '14. Thelma & Louise',
 '15. Eduardo Manostijeras',
 '16. La bella y la bestia',
 '17. Misery',
 '18. Los chicos del barrio',
 '19. El padrino parte III',
 '20. Aladdín',
 '21. La caza del Octubre Rojo',
 '22. JFK: Caso abierto',
 '23. Glengarry Glen Ross: Éxito a cualquier precio',
 '24. Braindead: tu madre se ha comido a mi perro',
 '25. Despertares']

In [11]:
# Lists to store numbers and names
movie_nr = []
title = []

# Iterate over each line and extract the number and name
for x in movies_names_filtered:
    n= x.split(". ")[0]
    t= x.split(". ")[-1]
    movie_nr.append(n)
    title.append(t)

In [12]:
movie_nr

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25']

In [13]:
title

['El silencio de los corderos',
 'Uno de los nuestros',
 'Terminator 2: El juicio final',
 'Mi primo Vinny',
 'Esencia de mujer',
 'Reservoir Dogs',
 'Tomates verdes fritos',
 'El último mohicano',
 'Bailando con lobos',
 'Desafío total',
 'Algunos hombres buenos',
 'Solo en casa',
 'Sin perdón',
 'Thelma & Louise',
 'Eduardo Manostijeras',
 'La bella y la bestia',
 'Misery',
 'Los chicos del barrio',
 'El padrino parte III',
 'Aladdín',
 'La caza del Octubre Rojo',
 'JFK: Caso abierto',
 'Glengarry Glen Ross: Éxito a cualquier precio',
 'Braindead: tu madre se ha comido a mi perro',
 'Despertares']

In [14]:
year_released = soup.find_all('span', class_="sc-b189961a-8 kLaxqf dli-title-metadata-item")
year_released

[<span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">1991</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">1h 58m</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">18</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">1990</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">2h 25m</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">18</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">1991</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">2h 17m</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">18</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">1992</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">2h</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">T</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-item">1992</span>,
 <span class="sc-b189961a-8 kLaxqf dli-title-metadata-it

In [15]:
year = []
age_rating = []
runtime = []

def extract_text_from_span(span_element):
    return span_element.get_text()

for i in range(0, len(year_released), 3):
    year_extract = year_released[i]
    runtime_extract = year_released[i+1]
    age_rating_extract = year_released[i+2]
    year.append(extract_text_from_span(year_extract))
    age_rating.append(extract_text_from_span(age_rating_extract))
    runtime.append(extract_text_from_span(runtime_extract))

In [16]:
year

['1991',
 '1990',
 '1991',
 '1992',
 '1992',
 '1992',
 '1991',
 '1992',
 '1990',
 '1990',
 '1992',
 '1990',
 '1992',
 '1991',
 '1990',
 '1991',
 '1990',
 '1991',
 '1990',
 '1992',
 '1990',
 '1991',
 '1992',
 '1992',
 '1990']

In [17]:
age_rating

['18',
 '18',
 '18',
 'T',
 'A',
 '18',
 '13',
 '13',
 'A',
 '18',
 '13',
 'A',
 '18',
 '13',
 'A',
 'T',
 '18',
 '13',
 '18',
 'A',
 'A',
 '13',
 '13',
 '18',
 'A']

In [18]:
runtime

['1h 58m',
 '2h 25m',
 '2h 17m',
 '2h',
 '2h 36m',
 '1h 39m',
 '2h 10m',
 '1h 52m',
 '3h 1m',
 '1h 53m',
 '2h 18m',
 '1h 43m',
 '2h 10m',
 '2h 10m',
 '1h 45m',
 '1h 24m',
 '1h 47m',
 '1h 52m',
 '2h 42m',
 '1h 30m',
 '2h 15m',
 '3h 9m',
 '1h 40m',
 '1h 44m',
 '2h 1m']

In [19]:
movies_Votes = soup.find_all('span', class_="ipc-rating-star--voteCount")
movies_Votes

[<span class="ipc-rating-star--voteCount"> (<!-- -->1.6M<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->1.3M<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->1.2M<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->143K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->333K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->1.1M<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->85K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->190K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->294K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->357K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->290K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->658K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCount"> (<!-- -->439K<!-- -->)</span>,
 <span class="ipc-rating-star--voteCoun

In [20]:
votes = [m_counts.text.strip().strip('()') for m_counts in movies_Votes]
votes

['1.6M',
 '1.3M',
 '1.2M',
 '143K',
 '333K',
 '1.1M',
 '85K',
 '190K',
 '294K',
 '357K',
 '290K',
 '658K',
 '439K',
 '175K',
 '528K',
 '482K',
 '238K',
 '156K',
 '427K',
 '472K',
 '216K',
 '171K',
 '117K',
 '104K',
 '160K']

In [21]:
movies_rating = soup.find_all('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')
movies_rating

[<span aria-label="IMDb rating: 8.6" class="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating" data-testid="ratingGroup--imdb-rating"><svg class="ipc-icon ipc-icon--star-inline" fill="currentColor" height="24" role="presentation" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg"><path d="M12 20.1l5.82 3.682c1.066.675 2.37-.322 2.09-1.584l-1.543-6.926 5.146-4.667c.94-.85.435-2.465-.799-2.567l-6.773-.602L13.29.89a1.38 1.38 0 0 0-2.581 0l-2.65 6.53-6.774.602C.052 8.126-.453 9.74.486 10.59l5.147 4.666-1.542 6.926c-.28 1.262 1.023 2.26 2.09 1.585L12 20.099z"></path></svg>8.6<span class="ipc-rating-star--voteCount"> (<!-- -->1.6M<!-- -->)</span></span>,
 <span aria-label="IMDb rating: 8.7" class="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating" data-testid="ratingGroup--imdb-rating"><svg class="ipc-icon ipc-icon--star-inline" fill="currentColor" height="24" role="presentation" viewbox="0 0 24 24" width="2

In [22]:
rating = [span.text.strip() [:3] for span in movies_rating]
rating

['8.6',
 '8.7',
 '8.6',
 '7.6',
 '8.0',
 '8.3',
 '7.7',
 '7.6',
 '8.0',
 '7.5',
 '7.7',
 '7.7',
 '8.2',
 '7.6',
 '7.9',
 '8.0',
 '7.8',
 '7.8',
 '7.6',
 '8.0',
 '7.5',
 '8.0',
 '7.7',
 '7.5',
 '7.8']

In [23]:
# <div class="ipc-html-content-inner-div">

movies_description = soup.find_all('div', class_='ipc-html-content-inner-div')
movies_description

[<div class="ipc-html-content-inner-div" role="presentation">A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.</div>,
 <div class="ipc-html-content-inner-div" role="presentation">The story of Henry Hill and his life in the mafia, covering his relationship with his wife Karen and his mob partners Jimmy Conway and Tommy DeVito.</div>,
 <div class="ipc-html-content-inner-div" role="presentation">A cyborg, identical to the one who failed to kill Sarah Connor, must now protect her ten year old son John from an even more advanced and powerful cyborg.</div>,
 <div class="ipc-html-content-inner-div" role="presentation">Two New Yorkers accused of murder in rural Alabama while on their way back to college call in the help of one of their cousins, a loudmouth lawyer with no trial experience.</div>,
 <div class="ipc-html-content-inner-div" role="presentation">A prep school student need

In [24]:
description = [span.text.strip() for span in movies_description]
description

['A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.',
 'The story of Henry Hill and his life in the mafia, covering his relationship with his wife Karen and his mob partners Jimmy Conway and Tommy DeVito.',
 'A cyborg, identical to the one who failed to kill Sarah Connor, must now protect her ten year old son John from an even more advanced and powerful cyborg.',
 'Two New Yorkers accused of murder in rural Alabama while on their way back to college call in the help of one of their cousins, a loudmouth lawyer with no trial experience.',
 'A prep school student needing money agrees to "babysit" a blind man, but the job is not at all what he anticipated.',
 'When a simple jewelry heist goes horribly wrong, the surviving criminals begin to suspect that one of them is a police informant.',
 'Evelyn, an ordinary housewife, visits a nursing home and befriends the old lady Ninny. 

In [25]:
len(description)

25

In [26]:
data = {
    'Movie Nr': movie_nr,
    'Title': title,
    'Year': year,
    'Rating': rating,
    'Runtime': runtime,
    'Description': description,
    'Rating': rating,
    'Votes': votes
}
def movie_data(data):
    df = pd.DataFrame(data)
    return df

movie_data(data)

,Movie Nr,Title,Year,Rating,Runtime,Description,Votes
0,1,El silencio de los corderos,1991,8.6,1h 58m,A young F.B.I. cadet must receive the help of ...,1.6M
1,2,Uno de los nuestros,1990,8.7,2h 25m,The story of Henry Hill and his life in the ma...,1.3M
2,3,Terminator 2: El juicio final,1991,8.6,2h 17m,"A cyborg, identical to the one who failed to k...",1.2M
3,4,Mi primo Vinny,1992,7.6,2h,Two New Yorkers accused of murder in rural Ala...,143K
4,5,Esencia de mujer,1992,8.0,2h 36m,A prep school student needing money agrees to ...,333K
5,6,Reservoir Dogs,1992,8.3,1h 39m,When a simple jewelry heist goes horribly wron...,1.1M
6,7,Tomates verdes fritos,1991,7.7,2h 10m,"Evelyn, an ordinary housewife, visits a nursin...",85K
7,8,El último mohicano,1992,7.6,1h 52m,Three trappers protect the daughters of a Brit...,190K
8,9,Bailando con lobos,1990,8.0,3h 1m,"Lieutenant John Dunbar, assigned to a remote w...",294K
9,10,Desafío total,1990,7.5,1h 53m,When a man goes in to have virtual vacation me...,357K


# Comment
The following columns weren't scraped due to accessibility issues (i button):
Genre, Director, Stars, Gross.